<a href="https://colab.research.google.com/github/tinawong15/portends-greatness/blob/master/elections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd

In [2]:
!git clone https://<user>:<password>@github.com/tinawong15/portends-greatness

Cloning into 'portends-greatness'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 9), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [3]:
!ls portends-greatness

1976-2016-president.csv  countypres_2000-2016.csv  elections.ipynb  README.md


In [0]:
csv_path = "portends-greatness/countypres_2000-2016.csv"
# csv_path = os.path.join('countypres_2000-2016.csv') 
df = pd.read_csv(csv_path, sep="\t") 
# df

In [0]:
winners = df.sort_values(['year', 'state', 'county', 'candidatevotes'], ascending=(True, True, True, False)).groupby(['year', 'state', 'county']).head(1)

In [0]:
# winners

In [0]:
counties_vote = winners.sort_values(['state', 'county', 'year'], ascending=(True, True, True))
# counties_vote

In [0]:
counties_vote = counties_vote.drop(["FIPS", "version", "office", "state_po"], axis=1)
# counties_vote

In [0]:
counties_vote["location"] = counties_vote['county'].str.cat(counties_vote['state'], sep=" ")

In [0]:
counties_vote = counties_vote.drop(['state', 'county'], axis=1)
# counties_vote

In [0]:
# counties_vote['order'] = counties_vote.apply(lambda row: (row.year - 2000)//4, axis = 1)

In [0]:
# counties_vote

In [0]:
new_counties_vote = counties_vote.copy()
# new_counties_vote

In [0]:
new_counties_vote["next_vote"] = new_counties_vote.groupby('location')["party"].shift(-1)
# new_counties_vote

In [0]:
new_counties_vote["no_swing"] = new_counties_vote["next_vote"] == new_counties_vote["party"]
# new_counties_vote

In [16]:
swing = new_counties_vote.groupby('location')['no_swing'].apply(lambda x: (x == False).sum()).reset_index(name='count')
swing['swinginess'] = (swing['count'] - 1) / 4
swing = swing.drop(["count"], axis=1)
# swing[swing['location'] == 'Autauga Alabama']
swing.sort_values(['swinginess'], ascending=(True))

,location,swinginess
0,Abbeville South Carolina,0.00
1667,Lewis Idaho,0.00
1668,Lewis Kentucky,0.00
1669,Lewis Missouri,0.00
1670,Lewis New York,0.00
2731,Sullivan Indiana,0.00
1672,Lewis Washington,0.00
1673,Lewis West Virginia,0.00
1675,Lexington South Carolina,0.00
1676,Lexington Virginia,0.00


In [26]:
csv_path = "portends-greatness/1976-2016-president.csv"
states = pd.read_csv(csv_path)
print(states.shape)
states = states[states['year'] >= 2000]
print(states.shape)

(3740, 14)
(1698, 14)


In [27]:
state_winners = states.sort_values(['year', 'state', 'candidatevotes'], ascending=(True, True, False)).groupby(['year', 'state']).head(1)
state_winners

,year,state,state_po,state_fips,state_cen,state_ic,office,candidate,party,writein,candidatevotes,totalvotes,version,notes
2042,2000,Alabama,AL,1,63,41,US President,"Bush, George W.",republican,False,941173,1666272,20171015,NaN
2047,2000,Alaska,AK,2,94,81,US President,"Bush, George W.",republican,False,167398,285560,20171015,NaN
2055,2000,Arizona,AZ,4,86,61,US President,"Bush, George W.",republican,False,781652,1532016,20171015,NaN
2062,2000,Arkansas,AR,5,71,42,US President,"Bush, George W.",republican,False,472940,921781,20171015,NaN
2069,2000,California,CA,6,93,71,US President,"Gore, Al",democrat,False,5861203,10965822,20171015,NaN
2076,2000,Colorado,CO,8,84,62,US President,"Bush, George W.",republican,False,883748,1741368,20171015,NaN
2086,2000,Connecticut,CT,9,16,1,US President,"Gore, Al",democrat,False,816015,1459525,20171015,NaN
2093,2000,Delaware,DE,10,51,11,US President,"Gore, Al",democrat,False,180068,327529,20171015,NaN
2100,2000,District of Columbia,DC,11,53,55,US President,"Gore, Al",democrat,False,171923,201894,20171015,NaN
2106,2000,Florida,FL,12,59,43,US President,"Bush, George W.",republican,False,2912790,5963110,20171015,NaN


In [29]:
states_vote = state_winners.sort_values(['state', 'year'], ascending=(True, True))
states_vote = states_vote.drop(["state_fips", "notes", "state_cen", "state_ic", "writein", "version", "office", "state_po"], axis=1)
states_vote

,year,state,candidate,party,candidatevotes,totalvotes
2042,2000,Alabama,"Bush, George W.",republican,941173,1666272
2411,2004,Alabama,"Bush, George W.",republican,1176394,1883415
2728,2008,Alabama,"McCain, John",republican,1266546,2099819
3079,2012,Alabama,"Romney, Mitt",republican,1255925,2074338
3395,2016,Alabama,"Trump, Donald J.",republican,1318255,2123372
2047,2000,Alaska,"Bush, George W.",republican,167398,285560
2415,2004,Alaska,"Bush, George W.",republican,190889,312598
2732,2008,Alaska,"McCain, John",republican,193841,326197
3083,2012,Alaska,"Romney, Mitt",republican,164676,300495
3400,2016,Alaska,"Trump, Donald J.",republican,163387,318608


In [0]:
new_states_vote = states_vote.copy()
new_states_vote["next_vote"] = new_states_vote.groupby('state')["party"].shift(-1)
new_states_vote["no_swing"] = new_states_vote["next_vote"] == new_states_vote["party"]

In [31]:
swing_states = new_states_vote.groupby('state')['no_swing'].apply(lambda x: (x == False).sum()).reset_index(name='count')
swing_states['swinginess'] = (swing_states['count'] - 1) / 4
swing_states = swing_states.drop(["count"], axis=1)
swing_states.sort_values(['swinginess'], ascending=(True))

,state,swinginess
0,Alabama,0.00
24,Mississippi,0.00
26,Montana,0.00
30,New Jersey,0.00
32,New York,0.00
34,North Dakota,0.00
36,Oklahoma,0.00
37,Oregon,0.00
21,Massachusetts,0.00
39,Rhode Island,0.00
